In [89]:
from matplotlib.colors import ListedColormap # for grgphing decision boundaries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split

In [90]:
data_folder = 'data'
X_train = pd.read_csv(f'./{data_folder}/X_train.csv')
y_train = pd.read_csv(f'./{data_folder}/y_train.csv')
X_test = pd.read_csv(f'./{data_folder}/X_test.csv')
y_test = pd.read_csv(f'./{data_folder}/y_test.csv')

In [91]:
class Network:
    def __init__(self, sample_input, num_layers, neurons_per_layer):
        self.layers = self.create_layers(sample_input, num_layers, neurons_per_layer)
        self.weights = self.create_weights()
        self.biases = [0 for i in range(num_layers+1)]
        
    @classmethod
    def create_layers(self, sample_input, num_layers, neurons_per_layer):
        """ function to create hidden layers """
        input_layer = [0 for i in sample_input]
        output_layer = [0 for i in range(3)]
        layers = []
        layers.append(input_layer)
        # for num of layers
        for i in range(num_layers):
            # create a empty list for neurons
            current_layer = []
            # create a neurons
            for j in range(neurons_per_layer[i]):
                # neuron = self.create_neuron(0.0 ,f'hidden layer #{i+1}' )
                current_layer.append(0)
            # add list of neurons to list of layers
            layers.append(current_layer)
        layers.append(output_layer)
        print(layers)
        return layers

    def create_weights(self):
        weights = []
        layers = self.layers
        for i in range(len(layers)-1):
            # get the length of the current layer
            current_layer = len(layers[i])
            # get the length of the next layer
            next_layer = len(layers[i+1])
            # create a matrix and append to list using the lengths of the layers 
            weights.append(np.random.rand(current_layer,next_layer))
        return weights 
        
        
    def sigmoid(self, activation):
        return (1/(1+np.exp(-(activation))))
    
    def feed_forward(self, X):
        layers = self.layers
        layers[0] = X
        for i in range(len(layers)-1):
            activations = self.sigmoid(np.dot(layers[i], self.weights[i]) + self.biases[i])
            # pass the activations to the next layer
            layers[i+1] = activations.copy()
        print(layers)
        return (layers[-1], max(layers[-1]))
    
    
    def back_propagation(self, output, y):
        layers = self.layers
        weights = self.weights
        biases = self.biases
        print(biases)
#         print(f'biases{biases}')
#         print(f'layers: {layers}')
        for i in range(1, len(layers)):
            # reshape the output layer to multiply with previous layer
            activations = np.reshape(layers[-i], (-1,1)) 
            # select previous layer
            prev_activations = layers[-(i+1)]
            # select the weights
            weight = weights[-i]
            # select the bias
            bias = biases[-i]
            cost_function = 2 * (output-y)
            dc_dw = prev_activations * (self.sigmoid(np.array(activations))*(1+self.sigmoid(np.array(activations)))) * cost_function
#             print(f'here:{prev_activations}')
            dc_db = (self.sigmoid(np.array(activations))*(1+self.sigmoid(np.array(activations)))) * cost_function
            print(f'biases before: {biases[-i]}')
            biases[-i] = dc_db
            print(f'biases after: {biases[-i]}')
#             print(-i)
            print()
#             print(f'weights before:{weights[-i]}')
#             weights[-i] = dc_dw
#             print(f'weights after:{weights[-i]}')
#             print(-i)
#             print()

        print()
        print(weights[-i])
        print()
        print(weights)
        return 0

In [92]:
X = [X_train.iloc[1]['sepal length (cm)'], X_train.iloc[1]['sepal width (cm)'], X_train.iloc[1]['petal length (cm)'], X_train.iloc[1]['petal length (cm)']]

In [93]:
new_class = Network(X,2, [4,3])

[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0], [0, 0, 0]]


In [94]:
last_layer, predicted = new_class.feed_forward(X)

[[4.8, 3.0, 1.4, 1.4], array([0.92547916, 0.99491442, 0.99564818, 0.990835  ]), array([0.75718639, 0.96080814, 0.90994711]), array([0.84846257, 0.80140217, 0.81734424])]


In [95]:
y = y_train.iloc[1]['class']
y

'Iris-setosa'

In [96]:
new_class.back_propagation(predicted, 1)

[0, 0, 0]
biases before: 0
biases after: [[-0.36083697]
 [-0.3536135 ]
 [-0.35606858]]

biases before: 0
biases after: [[-0.346764  ]
 [-0.37775939]
 [-0.3701584 ]]

biases before: 0
biases after: [[-0.37249062]
 [-0.38279612]
 [-0.38290393]
 [-0.38219634]]


[[0.08288466 0.52179188 0.70002917 0.49283326]
 [0.06266655 0.35685715 0.45058347 0.26934873]
 [0.48849138 0.51413625 0.25742038 0.18996076]
 [0.89249923 0.70091804 0.25751471 0.88826062]]

[array([[0.08288466, 0.52179188, 0.70002917, 0.49283326],
       [0.06266655, 0.35685715, 0.45058347, 0.26934873],
       [0.48849138, 0.51413625, 0.25742038, 0.18996076],
       [0.89249923, 0.70091804, 0.25751471, 0.88826062]]), array([[0.33169922, 0.95705308, 0.68170702],
       [0.16150081, 0.78286874, 0.67224283],
       [0.12678995, 0.9774711 , 0.06817763],
       [0.54844346, 0.56666179, 0.95412317]]), array([[0.11331513, 0.36278887, 0.73829557],
       [0.78757527, 0.37964134, 0.18621666],
       [0.9671814 , 0.83039969, 0.83577518]])]


0